In [1]:
import os
import cv2
import glob
import numpy as np
from master.models import *
from master.postprocess import *
from master.utils.chips import *

def chiparray(img_chip_path):
    _image = []
    for img_path in sorted(glob.glob(os.path.join(img_chip_path, "*.tif"))):
        with rasterio.open(img_path,'r') as f:
                imgr = np.moveaxis(f.read(),0,-1)
                _image.append(imgr)
                
    #Convert list to array       
    _image = np.array(_image).astype('float32')/255
    return _image

def pred_multi(output_folder):
    
    chips_folder = f'{out_fold}/chips' 
    chips_path = sorted(glob.glob(os.path.join(chips_folder,"*")))
    print(f'\nNumber of chips folder to predict: {len(chips_path)}')

    #Run cv prediction 
    results = []
    for i in range(len(chips_path)):
        with tf.device('/CPU:0'):
            pred = chiparray(chips_path[i])
        #print(pred.shape)
        model = load_model('./log/cv_mul/cv_multi.hdf5')
        pred_cv = model.predict(pred)#,1,verbose=1)
        results.append(pred_cv)

    #Project and merge the predicted chips into Planet scene
    #run projection of masks and save to a temporary folder
    with tf.device('/CPU:0'):
        tmp_path = f'./{output_folder}/tmp_pred'
        for i in range(len(chips_path)):
            proj_pred(chips_path[i],results[i],tmp_path,multi=True)

            #Run merge
            merge_masks(tmp_path,output_folder,chips_path[i])
            shutil.rmtree(chips_path[i])
        
    #cleanup chips folder
    os.rmdir(chips_folder)


In [2]:
#input folder containing raw planet SR scenes
#make sure that this folder only contains SR tif files
pred_fold = '../data/pred/raw_planet' 

#output folder for predicted scenes
out_fold = '../pred_out'
os.makedirs(out_fold,exist_ok=True)


In [3]:
#Preprocess image and extract chips
batch_chips(out_fold,pred_fold)
  

Number of images to run: 2
Processing 20230531_025957_07_242e_3B_AnalyticMS_SR_clip.tif .....
Processing 20230529_033209_85_248e_3B_AnalyticMS_SR_clip.tif .....


	Created 63 chips for 20230531_025957_07_242e_3B_AnalyticMS_SR_clip.tif
	Created 71 chips for 20230529_033209_85_248e_3B_AnalyticMS_SR_clip.tif


In [4]:
#predict chips, transform and mosaic to full scene
#tf warnings are due to use of only CPUs for this demo
#use GPU for faster large-scale processing
pred_multi(out_fold)


Number of chips folder to predict: 2


2024-03-26 18:10:44.627705: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-03-26 18:10:44.627746: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ceewater-cpu005): /proc/driver/nvidia/version does not exist
2024-03-26 18:10:44.628328: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


2/2 [==============================] - 8s 4s/step

Created 20230529_033209_85_248e_3B_AnalyticMS_SR_clip_mask in output folder

Created 20230531_025957_07_242e_3B_AnalyticMS_SR_clip_mask in output folder
